In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

from keras.layers import Input, Lambda, merge, Dense, Flatten, Embedding, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

import preprocessing_data as data
import metrics

Using TensorFlow backend.


## Load and transform data
We're going to load the Movielens $1M$ dataset and create triplets of (user, known positive item, randomly sampled negative item).

The success metric is AUC: in this case, the probability that a randomly chosen known positive item from the test set is ranked higher for a given user than a ranomly chosen negative item.

In [2]:
df = pd.read_csv('data/mldataset/ratings.dat', sep = '::', engine='python', header=None)
df.columns = ['UserId', 'MovieId', 'Rating', 'Timestamp']

x = np.array(df[['UserId', 'MovieId']])
y = np.array(df['Rating'])

# Read data
train, test = data.get_movielens_data()
num_users, num_items = max(df.UserId) +1, max(df.MovieId) +1

# Prepare the test triplets
test_uid, test_pid, test_nid = data.get_triplets(test)

In [7]:
train.toarray().shape

(944, 1683)

Define a metric between pairs, the _triplet loss function_.

## Neural Network Architecture

In [3]:
def identity_loss(y_true, y_pred): 

    return K.mean(y_pred - 0 * y_true)

def triplet_loss(inputs, alpha = 0.05):

    anchor, positive, negative = inputs
    
    pos_dist = K.sum(K.square(anchor-positive), axis=-1)
    neg_dist = K.sum(K.square(anchor-negative), axis=-1)
    loss = K.sum(K.maximum(pos_dist - neg_dist + alpha, 0), axis=0)

    return loss

def bpr_triplet_loss(inputs):

    anchor_latent, positive_item_latent, negative_item_latent = inputs

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(anchor_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(anchor_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss

def triploss(x): 
    res = tf.py_function(bpr_triplet_loss, [x], tf.float32)
    res.set_shape((None, 1))
    return res 

In [4]:
def getModel(n_users, n_items, emb_dim = 20, margin=1):
    
    # Input Layers
    user_input = Input(shape=[1], name = 'user_input')
    pos_item_input = Input(shape=[1], name = 'pos_item_input')
    neg_item_input = Input(shape=[1], name = 'neg_item_input')
    
    # Embedding Layers
    # Shared embedding layer for positive and negative items
    user_embedding = Embedding(output_dim=emb_dim, input_dim=n_users + 1, input_length=1, name='user_emb')(user_input)
    item_embedding = Embedding(output_dim=emb_dim, input_dim=n_items + 1, input_length=1, name='item_emb')
    
    pos_item_embedding = item_embedding(pos_item_input)
    neg_item_embedding = item_embedding(neg_item_input)
    
    user_vecs = Flatten(name='user_emb_vec')(user_embedding)
    pos_item_vecs = Flatten(name='pos_emb_vec')(pos_item_embedding)
    neg_item_vecs = Flatten(name='neg_emb_vec')(neg_item_embedding)
    
    # Triplet loss function 
    AP_loss = Lambda(lambda tensors:K.sum(K.square(tensors[0]*tensors[1]),axis=-1,keepdims=True),name='AP_loss')([user_vecs, pos_item_vecs])
    AN_loss = Lambda(lambda tensors:K.sum(K.square(tensors[0]*tensors[1]),axis=-1,keepdims=True),name='AN_loss')([user_vecs, neg_item_vecs])
    Triplet_loss = Lambda(lambda loss: 1.0 - K.sigmoid(loss[0] - loss[1]),
                      name='Triplet_loss')
    
    #call this layer on list of two input tensors.
    Final_loss = Triplet_loss([AP_loss, AN_loss])

    model = Model(inputs=[user_input, pos_item_input, neg_item_input],outputs=[Final_loss])
    model.compile(loss=identity_loss, optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [5]:
emb_dim = 100
n_epochs = 20

model = getModel(num_users, num_items, emb_dim)

# Print the model structure
print(model.summary())

# Sanity check, should be around 0.5
print('AUC before training %s' % metrics.full_auc(model, test))


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
pos_item_input (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
neg_item_input (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
user_emb (Embedding)            (None, 1, 100)       604200      user_input[0][0]                 
____________________________________________________________________________________________

In [6]:
for epoch in range(n_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = data.get_triplets(train)

    X = {
        'user_input': uid,
        'pos_item_input': pid,
        'neg_item_input': nid
    }

    model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              epochs=1,
              verbose=0,
              shuffle=True)

    print('AUC %s' % metrics.full_auc(model, test))

Epoch 0


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


AUC 0.4933859503570521
Epoch 1
AUC 0.4923985335323258
Epoch 2
AUC 0.4926312621200061
Epoch 3
AUC 0.4953075512386629
Epoch 4
AUC 0.49496981619833613
Epoch 5
AUC 0.4951794481094716
Epoch 6
AUC 0.495571400337752
Epoch 7
AUC 0.4966562351203805
Epoch 8
AUC 0.4957264756223382
Epoch 9
AUC 0.4967664864742963
Epoch 10
AUC 0.4988876970609935
Epoch 11
AUC 0.4986669394590502
Epoch 12
AUC 0.4983488105028148
Epoch 13
AUC 0.4979363490233906
Epoch 14
AUC 0.4980200197223901
Epoch 15
AUC 0.49754852043498443
Epoch 16
AUC 0.49775049834168994
Epoch 17
AUC 0.49821718098914103
Epoch 18
AUC 0.4987022261658519
Epoch 19
AUC 0.49949989949587953


In [7]:
model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              epochs=20,
              verbose=1,
              shuffle=True)

Epoch 1/20
49906/49906 [==============================] - 7s 137us/step - loss: 0.1102 - accuracy: 0.1033
Epoch 2/20
49906/49906 [==============================] - 7s 133us/step - loss: 0.1020 - accuracy: 0.0945
Epoch 3/20
49906/49906 [==============================] - 6s 129us/step - loss: 0.0949 - accuracy: 0.0867
Epoch 4/20
49906/49906 [==============================] - 7s 134us/step - loss: 0.0887 - accuracy: 0.0807
Epoch 5/20
49906/49906 [==============================] - 6s 124us/step - loss: 0.0830 - accuracy: 0.0758
Epoch 6/20
49906/49906 [==============================] - 6s 117us/step - loss: 0.0781 - accuracy: 0.0720
Epoch 7/20
49906/49906 [==============================] - 7s 137us/step - loss: 0.0738 - accuracy: 0.0686
Epoch 8/20
49906/49906 [==============================] - 6s 123us/step - loss: 0.0699 - accuracy: 0.0653
Epoch 9/20
49906/49906 [==============================] - 7s 138us/step - loss: 0.0665 - accuracy: 0.0628
Epoch 10/20
49906/49906 [=====================

In [9]:
df.iloc[235]

UserId               4
MovieId           2951
Rating               4
Timestamp    978294282
Name: 235, dtype: int64

In [21]:
metrics.predict(model, uid = 1, pids = 5)

-8.945791

In [17]:
pid

array([   1,    3,    6, ...,  928,  943, 1074], dtype=int32)